In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import json
import pandas as pd
from sqlalchemy import create_engine
import time
from selenium import webdriver
from datetime import datetime
from dateutil.parser import parse
import re
import numpy as np

In [2]:
chrome_path = 'envs\PythonData\chromedriver.exe'
driver = webdriver.Chrome(chrome_path)

In [3]:
# URL of page to be scraped
url = 'https://www.vgchartz.com/weekly/43464/USA'

In [4]:
# Retrieve page with the requests module
response = requests.get(url)
print(response)

<Response [200]>


In [5]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
# results are returned as an iterable list
chart_date_selector = soup.find('div', class_="chart_date_selector")

selections = chart_date_selector.find_all('option')


In [8]:
#scraping all rows in the main table
table = soup.findAll('table')[1]
#print(table)

rows = table.find_all('tr')
#print(rows)


In [11]:
value_list = []
date_list = []

#add date values from options into value_list
for selection in selections:

    values = selection['value']

    if int(values) >= 43058:

        dates = selection.text
        date_list.append(dates)

        value_list.append(values)
date_list.reverse()
print(date_list)


['18th Nov 2017', '25th Nov 2017', '02nd Dec 2017', '09th Dec 2017', '16th Dec 2017', '23rd Dec 2017', '30th Dec 2017', '06th Jan 2018', '13th Jan 2018', '20th Jan 2018', '27th Jan 2018', '03rd Feb 2018', '10th Feb 2018', '17th Feb 2018', '24th Feb 2018', '03rd Mar 2018', '10th Mar 2018', '17th Mar 2018', '24th Mar 2018', '31st Mar 2018', '07th Apr 2018', '14th Apr 2018', '21st Apr 2018', '28th Apr 2018', '05th May 2018', '12th May 2018', '19th May 2018', '26th May 2018', '02nd Jun 2018', '09th Jun 2018', '16th Jun 2018', '23rd Jun 2018', '30th Jun 2018', '07th Jul 2018', '14th Jul 2018', '21st Jul 2018', '28th Jul 2018', '04th Aug 2018', '11th Aug 2018', '18th Aug 2018', '25th Aug 2018', '01st Sep 2018', '08th Sep 2018', '15th Sep 2018', '22nd Sep 2018', '29th Sep 2018', '06th Oct 2018', '13th Oct 2018', '20th Oct 2018', '27th Oct 2018', '03rd Nov 2018', '10th Nov 2018', '17th Nov 2018', '24th Nov 2018', '01st Dec 2018', '08th Dec 2018', '15th Dec 2018', '22nd Dec 2018', '29th Dec 201

In [12]:
#getting urls for each week of data
url_list = []

usa = '/USA'

url2 = 'https://www.vgchartz.com/weekly/'

for y in value_list:

    date_url = url2 + y + usa
    
    url_list.append(date_url)
print(url_list)

['https://www.vgchartz.com/weekly/43464/USA', 'https://www.vgchartz.com/weekly/43457/USA', 'https://www.vgchartz.com/weekly/43450/USA', 'https://www.vgchartz.com/weekly/43443/USA', 'https://www.vgchartz.com/weekly/43436/USA', 'https://www.vgchartz.com/weekly/43429/USA', 'https://www.vgchartz.com/weekly/43422/USA', 'https://www.vgchartz.com/weekly/43415/USA', 'https://www.vgchartz.com/weekly/43408/USA', 'https://www.vgchartz.com/weekly/43401/USA', 'https://www.vgchartz.com/weekly/43394/USA', 'https://www.vgchartz.com/weekly/43387/USA', 'https://www.vgchartz.com/weekly/43380/USA', 'https://www.vgchartz.com/weekly/43373/USA', 'https://www.vgchartz.com/weekly/43366/USA', 'https://www.vgchartz.com/weekly/43359/USA', 'https://www.vgchartz.com/weekly/43352/USA', 'https://www.vgchartz.com/weekly/43345/USA', 'https://www.vgchartz.com/weekly/43338/USA', 'https://www.vgchartz.com/weekly/43331/USA', 'https://www.vgchartz.com/weekly/43324/USA', 'https://www.vgchartz.com/weekly/43317/USA', 'https://

In [13]:
game_data_list = []

for url in url_list:
    response2 = requests.get(url)
    soup = BeautifulSoup(response2.text, 'html.parser')
    table = soup.findAll('table')[1]
    rows = table.find_all('tr')
    #print(response2.text)
    #getting list of games
    for row in rows[1:]:
        #print(row)
        game_data = row.text
        #print(game_data)
        #game_data = re.sub(r"?\\n[0-9]","",game_data)
        splitting = game_data.rsplit('\n')
        game_data_list.append(splitting)
    #print(url)


In [11]:
df= pd.DataFrame(game_data_list)
cleaned_df = df.iloc[::2]
cleaned_df.drop([0, 2, 3, 4, 5, 7, 8, 9, 11, 12, 13], axis=1, inplace=True)
cleaned_df.rename(columns = {1:'Position', 6:'Video Game', 10:'Weekly Sales'}, inplace = True)
#cleaned_df.head(600)

In [12]:
cleaned_df.insert(3, "Week", 0)
cleaned_df

,Position,Video Game,Weekly Sales,Week
0,1,"Super Smash Bros. (2018) (NS)Nintendo, Fighting","432,869",0
2,2,Red Dead Redemption 2 (PS4)Take-Two Interactiv...,"215,113",0
4,3,"Call of Duty: Black Ops IIII (PS4)Activision, ...","198,626",0
6,4,Spider-Man (PS4) (PS4)Sony Interactive Enterta...,"167,154",0
8,5,"Mario Kart 8 Deluxe (NS)Nintendo, Racing","161,657",0
...,...,...,...,...
8840,71,BioShock The Collection (PS4)Take-Two Interact...,Pro,0
8842,72,"Tom Clancy's Rainbow Six: Siege (PS4)Ubisoft, ...",Pro,0
8844,73,"Forza Horizon 3 (XOne)Microsoft Game Studios, ...",Pro,0
8846,74,"The Evil Within II (PS4)Bethesda Softworks, Ac...",Pro,0


In [13]:
cleaned_df['Position']
week_list = []
for date in date_list:
    for position in cleaned_df['Position']:
        #print(position)
        week_list.append(date)
        if (int(position) != 0 and int (position) % 75 == 0):
            break 
cleaned_df['Week'] = week_list
cleaned_df

,Position,Video Game,Weekly Sales,Week
0,1,"Super Smash Bros. (2018) (NS)Nintendo, Fighting","432,869",18th Nov 2017
2,2,Red Dead Redemption 2 (PS4)Take-Two Interactiv...,"215,113",18th Nov 2017
4,3,"Call of Duty: Black Ops IIII (PS4)Activision, ...","198,626",18th Nov 2017
6,4,Spider-Man (PS4) (PS4)Sony Interactive Enterta...,"167,154",18th Nov 2017
8,5,"Mario Kart 8 Deluxe (NS)Nintendo, Racing","161,657",18th Nov 2017
...,...,...,...,...
8840,71,BioShock The Collection (PS4)Take-Two Interact...,Pro,29th Dec 2018
8842,72,"Tom Clancy's Rainbow Six: Siege (PS4)Ubisoft, ...",Pro,29th Dec 2018
8844,73,"Forza Horizon 3 (XOne)Microsoft Game Studios, ...",Pro,29th Dec 2018
8846,74,"The Evil Within II (PS4)Bethesda Softworks, Ac...",Pro,29th Dec 2018


In [14]:
split_df = cleaned_df["Video Game"].str.split("(", n=1, expand=True)

cleaned_df["Video Game"] = split_df[0]
#cleaned_df.to_csv("Video_Games_Cleaned.csv")
cleaned_df

,Position,Video Game,Weekly Sales,Week
0,1,Super Smash Bros.,"432,869",18th Nov 2017
2,2,Red Dead Redemption 2,"215,113",18th Nov 2017
4,3,Call of Duty: Black Ops IIII,"198,626",18th Nov 2017
6,4,Spider-Man,"167,154",18th Nov 2017
8,5,Mario Kart 8 Deluxe,"161,657",18th Nov 2017
...,...,...,...,...
8840,71,BioShock The Collection,Pro,29th Dec 2018
8842,72,Tom Clancy's Rainbow Six: Siege,Pro,29th Dec 2018
8844,73,Forza Horizon 3,Pro,29th Dec 2018
8846,74,The Evil Within II,Pro,29th Dec 2018


In [15]:
video_df = pd.read_csv("YT_data\Videos_Cleaned.csv")
video_df

,video_id,Unnamed: 1,0
0,lsfzA7sWlOM,0,nintendo
1,lsfzA7sWlOM,1,"""play"""
2,lsfzA7sWlOM,2,"""play nintendo"""
3,lsfzA7sWlOM,3,"""game"""
4,lsfzA7sWlOM,4,"""gameplay"""
...,...,...,...
14713,Xr2rgT9uEnA,7,"""lol"""
14714,ooyjaVdt-jA,0,call of duty
14715,ooyjaVdt-jA,1,"""cod"""
14716,ooyjaVdt-jA,2,"""activision"""


In [16]:
#Regex string matching

cleaned_df['join'] = 1
video_df['join'] = 1

dfFull = cleaned_df.merge(video_df, on='join').drop('join', axis=1)
video_df.drop('join', axis=1, inplace=True)

dfFull['match'] = dfFull.apply(lambda x: x["Video Game"].lower().find(x["0"].lower()), axis=1).ge(0)

In [17]:
narrow_games = dfFull.loc[dfFull['match'] == True]
narrow_games

,Position,Video Game,Weekly Sales,Week,video_id,Unnamed: 1,0,match
5075,1,Super Smash Bros.,"432,869",18th Nov 2017,JKP-rOESfMs,0,Super Smash Bros,True
5155,1,Super Smash Bros.,"432,869",18th Nov 2017,JKP-rOESfMs,0,Super Smash Bros,True
5235,1,Super Smash Bros.,"432,869",18th Nov 2017,JKP-rOESfMs,0,Super Smash Bros,True
5315,1,Super Smash Bros.,"432,869",18th Nov 2017,JKP-rOESfMs,0,Super Smash Bros,True
5395,1,Super Smash Bros.,"432,869",18th Nov 2017,JKP-rOESfMs,0,Super Smash Bros,True
...,...,...,...,...,...,...,...,...
65119616,75,New Super Mario Bros. 2,Pro,29th Dec 2018,b8GGWHGkx3Y,0,new,True
65119688,75,New Super Mario Bros. 2,Pro,29th Dec 2018,b8GGWHGkx3Y,0,new,True
65119760,75,New Super Mario Bros. 2,Pro,29th Dec 2018,b8GGWHGkx3Y,0,new,True
65119832,75,New Super Mario Bros. 2,Pro,29th Dec 2018,b8GGWHGkx3Y,0,new,True


In [18]:
new_narrow_df = narrow_games.drop_duplicates(subset='video_id', keep="last")
new_narrow_df

,Position,Video Game,Weekly Sales,Week,video_id,Unnamed: 1,0,match
6772221,11,Spyro Reignited Trilogy,"96,985",30th Dec 2017,3mB2fIt7vzw,0,IGN,True
6773124,11,Spyro Reignited Trilogy,"96,985",30th Dec 2017,4uvDgYKLgcU,0,IGN,True
6773541,11,Spyro Reignited Trilogy,"96,985",30th Dec 2017,ZWMp9BpWODk,0,IGN,True
6774519,11,Spyro Reignited Trilogy,"96,985",30th Dec 2017,bOG0X1WvXVU,0,IGN,True
13304735,4,Assassin's Creed Odyssey,"218,645",10th Feb 2018,s_SJZSAtLBA,0,Assassin's Creed Odyssey,True
39741963,1,God of War,"1,127,294",28th Jul 2018,K0u_kAWLJOA,0,God of War,True
59292742,54,Fortnite,Pro,24th Nov 2018,igczJLuaelg,0,Fortnite,True
59293620,54,Fortnite,Pro,24th Nov 2018,sq4hYZlDPec,0,fortnite,True
59294522,54,Fortnite,Pro,24th Nov 2018,fDRrqJU4-Tk,0,Fortnite,True
59298415,54,Fortnite,Pro,24th Nov 2018,qPEIPu0V8GQ,0,Fortnite,True


In [19]:
final_videogames_df = new_narrow_df[['Position', 'Video Game', 'Weekly Sales', 'Week', 'video_id']]
final_videogames_df

,Position,Video Game,Weekly Sales,Week,video_id
6772221,11,Spyro Reignited Trilogy,"96,985",30th Dec 2017,3mB2fIt7vzw
6773124,11,Spyro Reignited Trilogy,"96,985",30th Dec 2017,4uvDgYKLgcU
6773541,11,Spyro Reignited Trilogy,"96,985",30th Dec 2017,ZWMp9BpWODk
6774519,11,Spyro Reignited Trilogy,"96,985",30th Dec 2017,bOG0X1WvXVU
13304735,4,Assassin's Creed Odyssey,"218,645",10th Feb 2018,s_SJZSAtLBA
39741963,1,God of War,"1,127,294",28th Jul 2018,K0u_kAWLJOA
59292742,54,Fortnite,Pro,24th Nov 2018,igczJLuaelg
59293620,54,Fortnite,Pro,24th Nov 2018,sq4hYZlDPec
59294522,54,Fortnite,Pro,24th Nov 2018,fDRrqJU4-Tk
59298415,54,Fortnite,Pro,24th Nov 2018,qPEIPu0V8GQ


In [20]:
rds_connection_string = "postgres:norocel7@database-1.cgiopwhezyby.us-east-2.rds.amazonaws.com/video_games_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

engine.table_names()

final_videogames_df.to_sql(name='video_games', con=engine, if_exists='replace', index=False)